In [1]:
# Importação de Bibliotécas
import pandas as pd
import json
import re
import numpy as np
import unicodedata
from datetime import datetime

In [2]:
# Definindo caminho do arquivo
file_path = 'project_data_copy.json'

In [3]:
# Carregar o arquivo JSON
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [4]:
# Convertendo para DataFrame
df_projects = pd.DataFrame(data)

In [5]:
# Display da Tabela / Df
df_projects

,link,title,status,uf,setores,ods,identificador_do_projeto,ministerio,secretaria,resolucoes,...,etapas,reuniao,data_qualificacao,modalidade_operacional,detalhamento_modalidade_operacional,extensão,capex,leilao,ano_leilao,projeto_ativo
0,https://ppi.gov.br/projetos/barragem-de-morrin...,Barragem de Morrinhos (DNOCS),[Em andamento],[BA],"[Infraestrutura Hídrica, Barragens]",[],691,None,SIEC,[],...,"[{'etapa': 'Contratação de estruturador', 'sta...","[28ª Reunião, -14.57186231387606, -40.30083183...",20250325,Concessão/Autorização,Concessão/Autorização,None,None,Estimado,None,Ativo e divulgado
1,https://ppi.gov.br/projetos/barragem-de-champr...,Barragem de Champrão (DNOCS),[Em andamento],[BA],"[Infraestrutura Hídrica, Barragens]",[],690,None,SIEC,[],...,"[{'etapa': 'Contratação de estruturador', 'sta...","[28ª Reunião, -14.913505071768713, -41.9673155...",20250325,Concessão/Autorização,Concessão/Autorização,None,None,Estimado,None,Ativo e divulgado
2,https://ppi.gov.br/projetos/barragem-de-truvis...,Barragem de Truvisco (DNOCS),[Em andamento],[BA],"[Infraestrutura Hídrica, Barragens]",[],689,None,SIEC,[],...,"[{'etapa': 'Contratação de estruturador', 'sta...","[28ª Reunião, -14.546831088064035, -42.3207417...",20250325,Concessão/Autorização,Concessão/Autorização,None,None,Estimado,None,Ativo e divulgado
3,https://ppi.gov.br/projetos/barragem-de-tremed...,Barragem de Tremedal (DNOCS),[Em andamento],[BA],"[Infraestrutura Hídrica, Barragens]",[],688,None,SIEC,[],...,"[{'etapa': 'Contratação de estruturador', 'sta...","[28ª Reunião, -14.989697782965765, -41.4071464...",20250325,Concessão/Autorização,Concessão/Autorização,None,None,Estimado,None,Ativo e divulgado
4,https://ppi.gov.br/projetos/barragem-de-anage-...,Barragem de Anagé (DNOCS),[Em andamento],[BA],"[Infraestrutura Hídrica, Barragens]",[],687,None,SIEC,[],...,"[{'etapa': 'Contratação de estruturador', 'sta...","[28ª Reunião, -14.684712426383692, -41.1078574...",20250325,Concessão/Autorização,Concessão/Autorização,None,None,Estimado,None,Ativo e divulgado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,https://ppi.gov.br/projetos/aeroporto-de-forta...,Aeroporto de Fortaleza/ CE,[Concluídos],[CE],"[Transportes, Aeroportuário]",[],6,MPOR,SIEC,"[[4378, Resolução CPPI 2, de 13 de setembro de...",...,"[{'etapa': 'Estudo', 'status': 'Concluído', 'd...",[1ª REUNIÃO],20160913,Concessão/Autorização,Concessão,None,"1370000000,00",Leiloado,2017,Ativo e divulgado
635,https://ppi.gov.br/projetos/aeroporto-de-flori...,Aeroporto de Florianópolis/ SC,[Concluídos],[SC],"[Transportes, Aeroportuário]",[],5,MPOR,SIEC,"[[4378, Resolução CPPI 2, de 13 de setembro de...",...,"[{'etapa': 'Estudo', 'status': 'Concluído', 'd...",[1ª REUNIÃO],20160913,Concessão/Autorização,Concessão,None,"960000000,00",Leiloado,2017,Ativo e divulgado
636,https://ppi.gov.br/projetos/aeroporto-de-salva...,Aeroporto de Salvador/ BA,[Concluídos],[BA],"[Transportes, Aeroportuário]",[],4,MPOR,SIEC,"[[4378, Resolução CPPI 2, de 13 de setembro de...",...,"[{'etapa': 'Estudo', 'status': 'Concluído', 'd...",[1ª REUNIÃO],20160913,Concessão/Autorização,Concessão,None,"2300000000,00",Leiloado,2017,Ativo e divulgado
637,https://ppi.gov.br/projetos/aeroporto-de-porto...,Aeroporto de Porto Alegre/ RS,[Concluídos],[RS],"[Transportes, Aeroportuário]",[],3,MPOR,SIEC,"[[4378, Resolução CPPI 2, de 13 de setembro de...",...,"[{'etapa': 'Estudo', 'status': 'Concluído', 'd...",[1ª REUNIÃO],20160913,Concessão/Autorização,Concessão,None,"1900000000,00",Leiloado,2017,Ativo e divulgado


In [6]:
# Mostrando as Colunas
df_projects.columns

Index(['link', 'title', 'status', 'uf', 'setores', 'ods',
       'identificador_do_projeto', 'ministerio', 'secretaria', 'resolucoes',
       'decretos', 'etapas', 'reuniao', 'data_qualificacao',
       'modalidade_operacional', 'detalhamento_modalidade_operacional',
       'extensão', 'capex', 'leilao', 'ano_leilao', 'projeto_ativo'],
      dtype='object')

---

---
<h1><i>Definição de Funções</i></h1>

In [7]:
# Função para normalizar texto, removendo acentos e convertendo para minúsculas
def normalize_text(text):
    if not isinstance(text, str):
        return ''
    # Normaliza os caracteres unicode para sua forma decomposta
    text = unicodedata.normalize('NFKD', text)
    # Remove caracteres não ASCII (acentos)
    text = text.encode('ASCII', 'ignore').decode('ASCII')
    # Converte para minúsculas e remove espaços em branco extras
    return text.lower().strip()

In [8]:
# Função para remover símbolos do DataFrame
def remove_symbols(text):
    return re.sub(r"[{}\[\]']", "", str(text))

In [9]:
# Função para renomear as colunas
def rename_columns(df):
    def capitalize_column_name(name):
        return ' '.join([word.capitalize() for word in name.split('_')])
    df.columns = [capitalize_column_name(col) for col in df.columns]
    return df

In [10]:
# Função para limpar as resoluções e decretos
def clean_resolucoes_decretos(resolucoes_decretos):
    cleaned = []
    for res in resolucoes_decretos:
        cleaned.append(re.sub(r"\['\d+', '", "", str(res)).replace("']", ""))
    return ', '.join(cleaned)

In [11]:
# Verifica os projetos Favoritos (NOVO)
def verificar_prioritario(status_list):
    """
    Verifica se a lista de status contém "prioritário".
    Retorna "Sim" se houver, senão retorna "Não".
    """
    return "Sim" if "prioritário" in (status_list or []) else "Não"

In [12]:
# Função para limpar a coluna 'Extensão' e deixar apenas os valores numéricos
def clean_extensao(extensao):
    if isinstance(extensao, str):
        # Usamos expressão regular para remover todos os caracteres não numéricos e não ponto
        extensao = re.sub(r'[^\d,\.]', '', extensao)
        # Substituir vírgula por ponto para manter o formato numérico correto
        extensao = extensao.replace(',', '.')
        try:
            # Tentar converter para float (numérico)
            return float(extensao)
        except ValueError:
            return np.nan  # Se não for possível converter, retorna NaN
    return extensao

In [13]:
# Função importante que separa as ETAPAS em seu padrão de progresso
def split_etapas(etapas, secretaria, status_geral, etapa_final='Contrato'):
    etapas_flat = {}
    prioritario = verificar_prioritario(status_geral)  # Verifica se é prioritário

    # Se o status geral indicar "Concluídos", marcar tudo como concluído
    if status_geral == ['Concluídos']:
        for etapa in etapas:
            etapa_nome = etapa.get('etapa', '').strip()
            etapas_flat[f'{etapa_nome} (status)'] = 'Concluído'
            etapas_flat[f'Data {etapa_nome} (início)'] = etapa.get('data_inicio', '').strip() or None
            etapas_flat[f'Data {etapa_nome} (fim)'] = etapa.get('data_fim', '').strip() or None
        etapas_flat['Status do projeto'] = 'Concluído'
        etapas_flat['Etapa'] = 'Concluído'
        etapas_flat['Prioritário'] = prioritario  # Adiciona a coluna prioritário
        return etapas_flat

    # Caso contrário, processar etapas normalmente
    etapa_atual = None
    etapa_anterior_concluida = False

    for etapa in etapas:
        etapa_nome = etapa.get('etapa', '').strip()
        status = etapa.get('status', '').strip()

        # Normalização para verificar status
        status_normalized = normalize_text(status)

        # Se "Contrato" estiver concluído, tudo anterior é "Concluído"
        if etapa_nome == etapa_final and (status_normalized == 'concluido' or status == '- Selecionar -'):
            etapa_anterior_concluida = True

        # Ignorar "Desconhecido" e definir próxima etapa válida como atual
        if status_normalized == 'desconhecido':
            continue

        # Condição para SISU, ignorar "Acórdão TCU"
        if secretaria == 'SISU' and etapa_nome == 'Acórdão TCU':
            continue

        # Atualizar etapa atual se ainda não definida
        if not etapa_atual and status_normalized != 'concluido':
            etapa_atual = etapa_nome

        # Salvar valores originais
        etapas_flat[f'{etapa_nome} (status)'] = 'Concluído' if etapa_anterior_concluida else status
        etapas_flat[f'Data {etapa_nome} (início)'] = etapa.get('data_inicio', '').strip() or None
        etapas_flat[f'Data {etapa_nome} (fim)'] = etapa.get('data_fim', '').strip() or None

    # Tratar casos onde o status final está vazio
    if not any(etapa.get('status', '').strip() for etapa in etapas):
        etapas_flat['Status do projeto'] = 'Em Andamento'
        etapas_flat['Etapa'] = etapa_atual if etapa_atual else 'Desconhecido'
    elif etapa_anterior_concluida or all(normalize_text(etapa.get('status', '')) == 'concluido' for etapa in etapas):
        etapas_flat['Status do projeto'] = 'Concluído'
        etapas_flat['Etapa'] = 'Concluído'
    else:
        etapas_flat['Status do projeto'] = 'Em Andamento'
        etapas_flat['Etapa'] = etapa_atual if etapa_atual else 'Desconhecido'

    etapas_flat['Prioritário'] = prioritario  # Adiciona a coluna prioritário
    return etapas_flat

In [14]:
# Função para tratar a coluna 'reuniao'
def clean_reuniao(reuniao):
    if isinstance(reuniao, str):
        return reuniao.split(',', 1)[0].strip()  # Mantém apenas o texto antes da primeira vírgula
    return reuniao

In [15]:
# Função que formata e ordena padrão de Reunião
def reuniao_format(name):
    if not isinstance(name, str):
        return None  # Se não for string, retorna None (ou outra lógica que preferir)
    number = ''.join(filter(str.isdigit, name))
    if 'CFEP' in name or 'FEP' in name:
        return f'{number}ª Reunião FEP'
    else:
        return f'{number}ª Reunião'

---

In [16]:
# Dividir a coluna 'setores' em 'setor' e 'subsetor', preenchendo valores ausentes
df_projects['setores'] = df_projects['setores'].astype(str)  # Convertendo todos os valores para strings
setores_split = df_projects['setores'].str.split(',', n=1, expand=True)
df_projects['setor'] = setores_split[0].str.strip()
df_projects['subsetor'] = setores_split[1].str.strip().fillna('')

In [17]:
# Substituir os nomes dos ministérios
ministerio_substi = {
    'Ministério da Defesa': 'MD',
    'Ministério dos Transportes': 'MT'
}
df_projects['ministerio'] = df_projects['ministerio'].replace(ministerio_substi)

In [18]:
# Limpar as resoluções e decretos
df_projects['resolucoes'] = df_projects['resolucoes'].apply(clean_resolucoes_decretos)
df_projects['decretos'] = df_projects['decretos'].apply(clean_resolucoes_decretos)

In [19]:
# Aplicar a função para dividir as etapas
etapas_cols_df = df_projects.apply(lambda row: split_etapas(row['etapas'], row['secretaria'], row['status']), axis=1).apply(pd.Series)

# Combinar as novas colunas ao DataFrame original
df_projects = pd.concat([df_projects.drop(columns=['etapas']), etapas_cols_df], axis=1)

In [20]:
# Aplicando funbção que remove os simbolos advindos do JSON como ':', ',' e chaves []
df_projects = df_projects.applymap(remove_symbols)

C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_15148\1612927762.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_projects = df_projects.applymap(remove_symbols)


In [21]:
# Aplicar a função para limpar a coluna 'reuniao'
df_projects['reuniao'] = df_projects['reuniao'].apply(clean_reuniao)

In [22]:
# Substituir valores indesejados por NaN
df_projects.replace(['N/I', 'não informado', 'N/A', '- Selecionar -', 'None', ''], np.nan, inplace=True)

# Preencher NaNs com string vazia (se necessário)
df_projects.fillna('', inplace=True)

In [23]:
# Dicionário de mapeamento para substituir os valores na coluna 'status'
mapeamento = {
    r'.*Concluídos.*': 'Concluídos',
    r'.*Suspenso.*': 'Suspensos',
    r'.*(Dataroom Aberto|Editais Abertos|Em andamento).*': 'Em Aberto'
}

In [24]:
# Usando replace para mapear os valores conforme o dicionário
df_projects['status'] = df_projects['status'].replace(mapeamento, regex=True)

In [25]:
# Renomear colunas
df_projects = rename_columns(df_projects)

In [26]:
#Display no Dataframe
df_projects

,Link,Title,Status,Uf,Setores,Ods,Identificador Do Projeto,Ministerio,Secretaria,Resolucoes,...,Data publicação parecer (fim),Termo aditivo (status),Data termo aditivo (início),Data termo aditivo (fim),Audiência pública (status),Data audiência pública (início),Data audiência pública (fim),Licenciamento (status),Data licenciamento (início),Data licenciamento (fim)
0,https://ppi.gov.br/projetos/barragem-de-morrin...,Barragem de Morrinhos (DNOCS),Em Aberto,BA,"Infraestrutura Hídrica, Barragens",,691,,SIEC,,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,https://ppi.gov.br/projetos/barragem-de-champr...,Barragem de Champrão (DNOCS),Em Aberto,BA,"Infraestrutura Hídrica, Barragens",,690,,SIEC,,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,https://ppi.gov.br/projetos/barragem-de-truvis...,Barragem de Truvisco (DNOCS),Em Aberto,BA,"Infraestrutura Hídrica, Barragens",,689,,SIEC,,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,https://ppi.gov.br/projetos/barragem-de-tremed...,Barragem de Tremedal (DNOCS),Em Aberto,BA,"Infraestrutura Hídrica, Barragens",,688,,SIEC,,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,https://ppi.gov.br/projetos/barragem-de-anage-...,Barragem de Anagé (DNOCS),Em Aberto,BA,"Infraestrutura Hídrica, Barragens",,687,,SIEC,,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,https://ppi.gov.br/projetos/aeroporto-de-forta...,Aeroporto de Fortaleza/ CE,Concluídos,CE,"Transportes, Aeroportuário",,6,MPOR,SIEC,"Resolução CPPI 2, de 13 de setembro de 2016, R...",...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
635,https://ppi.gov.br/projetos/aeroporto-de-flori...,Aeroporto de Florianópolis/ SC,Concluídos,SC,"Transportes, Aeroportuário",,5,MPOR,SIEC,"Resolução CPPI 2, de 13 de setembro de 2016, R...",...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
636,https://ppi.gov.br/projetos/aeroporto-de-salva...,Aeroporto de Salvador/ BA,Concluídos,BA,"Transportes, Aeroportuário",,4,MPOR,SIEC,"Resolução CPPI 2, de 13 de setembro de 2016, R...",...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
637,https://ppi.gov.br/projetos/aeroporto-de-porto...,Aeroporto de Porto Alegre/ RS,Concluídos,RS,"Transportes, Aeroportuário",,3,MPOR,SIEC,"Resolução CPPI 2, de 13 de setembro de 2016, R...",...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [27]:
df_projects.columns

Index(['Link', 'Title', 'Status', 'Uf', 'Setores', 'Ods',
       'Identificador Do Projeto', 'Ministerio', 'Secretaria', 'Resolucoes',
       'Decretos', 'Reuniao', 'Data Qualificacao', 'Modalidade Operacional',
       'Detalhamento Modalidade Operacional', 'Extensão', 'Capex', 'Leilao',
       'Ano Leilao', 'Projeto Ativo', 'Setor', 'Subsetor',
       'Contratação de estruturador (status)',
       'Data contratação de estruturador (início)',
       'Data contratação de estruturador (fim)', 'Estudo (status)',
       'Data estudo (início)', 'Data estudo (fim)',
       'Consulta pública (status)', 'Data consulta pública (início)',
       'Data consulta pública (fim)', 'Órgão de controle (status)',
       'Data órgão de controle (início)', 'Data órgão de controle (fim)',
       'Edital (status)', 'Data edital (início)', 'Data edital (fim)',
       'Leilão de projeto (status)', 'Data leilão de projeto (início)',
       'Data leilão de projeto (fim)', 'Status do projeto', 'Etapa',
       'P

In [28]:
# Realocação de colunas e renomeação adicional
df_projects.rename(columns={'Uf': 'UF'}, inplace=True)
df_projects.rename(columns={'Ods': 'ODS'}, inplace=True)

df_projects = df_projects[['Identificador Do Projeto', 'Title', 'Status', 'UF', 'ODS',
       'Ministerio', 'Secretaria', 'Setor', 'Subsetor', 'Resolucoes',
       'Decretos', 'Reuniao', 'Data Qualificacao', 'Modalidade Operacional',
       'Detalhamento Modalidade Operacional', 'Extensão', 'Capex', 'Leilao', 'Ano Leilao',
       'Projeto Ativo', 'Estudo (status)',
       'Data estudo (início)', 'Data estudo (fim)',
       'Consulta pública (status)', 'Data consulta pública (início)',
       'Data consulta pública (fim)', 'Acórdão tcu (status)',
       'Data acórdão tcu (início)', 'Data acórdão tcu (fim)',
       'Edital (status)', 'Data edital (início)', 'Data edital (fim)',
       'Leilão de projeto (status)', 'Data leilão de projeto (início)',
       'Data leilão de projeto (fim)', 'Contrato (status)',
       'Data contrato (início)', 'Data contrato (fim)', 'Publicação parecer (status)',
       'Data publicação parecer (início)', 'Data publicação parecer (fim)',
       'Termo aditivo (status)', 'Data termo aditivo (início)',
       'Data termo aditivo (fim)', 'Órgão de controle (status)',
       'Data órgão de controle (início)', 'Data órgão de controle (fim)',
       'Licenciamento (status)',
       'Data licenciamento (início)', 'Data licenciamento (fim)',
       'Audiência pública (status)', 'Data audiência pública (início)',
       'Data audiência pública (fim)', 'Status do projeto', 'Prioritário',
       'Etapa', 'Link']]

In [29]:
# Aplicar a função para limpar a coluna 'Extensão'
df_projects['Extensão'] = df_projects['Extensão'].apply(clean_extensao)

C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_15148\1015900349.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_projects['Extensão'] = df_projects['Extensão'].apply(clean_extensao)


In [30]:
# Formatação de valores financeiros
financial_columns = ['Capex']

for col in financial_columns:
    # Primeiro, removemos os pontos para eliminar os separadores de milhares
    df_projects[col] = df_projects[col].replace(r'\.', '', regex=True)
    # Depois, substituímos as vírgulas por pontos para o formato decimal correto
    df_projects[col] = df_projects[col].replace(',', '.', regex=True)
    # Convertendo para numérico
    df_projects[col] = pd.to_numeric(df_projects[col], errors='coerce')

C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_15148\1640090943.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_projects[col] = df_projects[col].replace(r'\.', '', regex=True)
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_15148\1640090943.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_projects[col] = df_projects[col].replace(',', '.', regex=True)
C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_15148\1640090943.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [31]:
# Função para corrigir a coluna 'Reuniao'
def correct_reuniao(reuniao):
    if isinstance(reuniao, str):
        cleaned_value = reuniao.split(',', 1)[0].strip()  # Mantém apenas o texto antes da primeira vírgula e remove espaços em branco
        if cleaned_value in ['ª Reunião', 'ª Reunião FEP']:
            return 'Projeto FEP'
        # Verificar se há mais de dois números na reunião
        digits_only = ''.join(filter(str.isdigit, cleaned_value))
        if len(digits_only) > 2:
            return np.nan
        return cleaned_value
    return reuniao

# Aplicar a função para corrigir a coluna 'Reuniao'
df_projects['Reuniao'] = df_projects['Reuniao'].apply(correct_reuniao)

C:\Users\lucas.bruno\AppData\Local\Temp\ipykernel_15148\4102791967.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_projects['Reuniao'] = df_projects['Reuniao'].apply(correct_reuniao)


In [32]:
# Mostrando as colunas Finais
df_projects.columns

Index(['Identificador Do Projeto', 'Title', 'Status', 'UF', 'ODS',
       'Ministerio', 'Secretaria', 'Setor', 'Subsetor', 'Resolucoes',
       'Decretos', 'Reuniao', 'Data Qualificacao', 'Modalidade Operacional',
       'Detalhamento Modalidade Operacional', 'Extensão', 'Capex', 'Leilao',
       'Ano Leilao', 'Projeto Ativo', 'Estudo (status)',
       'Data estudo (início)', 'Data estudo (fim)',
       'Consulta pública (status)', 'Data consulta pública (início)',
       'Data consulta pública (fim)', 'Acórdão tcu (status)',
       'Data acórdão tcu (início)', 'Data acórdão tcu (fim)',
       'Edital (status)', 'Data edital (início)', 'Data edital (fim)',
       'Leilão de projeto (status)', 'Data leilão de projeto (início)',
       'Data leilão de projeto (fim)', 'Contrato (status)',
       'Data contrato (início)', 'Data contrato (fim)',
       'Publicação parecer (status)', 'Data publicação parecer (início)',
       'Data publicação parecer (fim)', 'Termo aditivo (status)',
       

---

In [33]:
# Salvar os dados coletados em um arquivo Excel
df_projects.to_excel('Projects_Cleaned.xlsx', index=False, sheet_name='Projetos')